# Referendum Results

This notebook scrapes the voting results in the membership of the European Union on 23rd June, 2016. This is scraped from the BBC website

In [1]:
# Check Python version for compatibility/reference
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/usr/local/anaconda3/bin/python
3.8.3 (default, Jul  2 2020, 11:26:31) 
[Clang 10.0.0 ]
sys.version_info(major=3, minor=8, micro=3, releaselevel='final', serial=0)


In [2]:
# Import requisite libraries for analysis
from collections import defaultdict
from decimal import Decimal
import string
import requests
from bs4 import BeautifulSoup

In [3]:
# Set a constant for the alphabet's letters
ALPHABET = string.ascii_lowercase

In [4]:
# Set the base url in readiness to append the letter
BASE_URL = 'https://www.bbc.co.uk/news/politics/eu_referendum/results/local/'

In [5]:
# Set initial variables and results dictionary
results = defaultdict(list)
leave_votes, remain_votes = 0, 0

In [6]:
# Scrape content from each lettered page from the BBC website
# Print the status code returned from each page for review
for letter in ALPHABET:
    page_response = requests.get(f"{BASE_URL}{letter}", timeout=5)
    print(f"{'URL:'} {BASE_URL}{letter}{' | Status Code: '}{page_response.status_code}")
    if page_response:
        page_content = BeautifulSoup(page_response.content, "html.parser")
        areas = page_content.find_all('div', attrs={'class': 'eu-ref-result-bar'})
        for area in areas:
            results['area_name'].append(area.find('h3').getText())
            cleaned_leave_votes = int(area.find_all('div',
                                    {'class': 'eu-ref-result-bar__votes'}
                                    )[0].string.strip().split('\n')[0].strip().replace(',', ''))
            results['leave_votes'].append(cleaned_leave_votes)
            cleaned_remain_votes = int(area.find_all('div',
                                    {'class': 'eu-ref-result-bar__votes'}
                                    )[1].string.strip().split('\n')[0].strip().replace(',', ''))
            results['remain_votes'].append(cleaned_remain_votes)
            area_votes = cleaned_leave_votes + cleaned_remain_votes
            results['area_votes'].append(area_votes)
            results['leave_percent'].append(f"{cleaned_leave_votes / area_votes:.1%}")
            results['remain_percent'].append(f"{cleaned_remain_votes / area_votes:.1%}")
            results['turnout'].append(area.find(
                'div', {'class': 'eu-ref-result-bar__turnout'})
                .getText().replace('Turnout: ', '')
            )
            leave_votes = sum(Decimal(num) for num in results['leave_votes'])
            remain_votes = sum(Decimal(num) for num in results['remain_votes'])
    else:
        continue

URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/a | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/b | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/c | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/d | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/e | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/f | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/g | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/h | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/i | Status Code: 200
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/j | Status Code: 404
URL: https://www.bbc.co.uk/news/politics/eu_referendum/results/local/k | Status Code: 200
URL: https

In [7]:
# Print a totals summary for the number of leave and remain votes
print(f"Leave votes: {leave_votes:,}\nRemain votes: {remain_votes:,}")

Leave votes: 17,410,742
Remain votes: 16,141,241


In [8]:
# Zip together the individual components of an area's result
area_results = zip(
    results['area_name'],
    results['leave_votes'],
    results['leave_percent'],
    results['remain_votes'],
    results['remain_percent'],
    results['area_votes'],
    results['turnout'],
)

In [9]:
# Print the results for each area
results = []
for area in area_results:
    results.append(area)

In [10]:
results

[('Aberdeen City', 40729, '38.9%', 63985, '61.1%', 104714, '67.9%'),
 ('Aberdeenshire', 62516, '45.0%', 76445, '55.0%', 138961, '70.6%'),
 ('Adur', 20315, '54.6%', 16914, '45.4%', 37229, '76.4%'),
 ('Allerdale', 31809, '58.6%', 22429, '41.4%', 54238, '72.9%'),
 ('Amber Valley', 44501, '60.3%', 29319, '39.7%', 73820, '76.3%'),
 ('Angus', 26511, '44.7%', 32747, '55.3%', 59258, '68.0%'),
 ('Argyll & Bute', 19202, '39.4%', 29494, '60.6%', 48696, '73.1%'),
 ('Arun', 56936, '62.5%', 34193, '37.5%', 91129, '77.8%'),
 ('Ashfield', 46720, '69.8%', 20179, '30.2%', 66899, '72.8%'),
 ('Ashford', 41472, '59.4%', 28314, '40.6%', 69786, '77.1%'),
 ('Aylesbury Vale', 53956, '50.5%', 52877, '49.5%', 106833, '78.4%'),
 ('Babergh', 29933, '54.2%', 25309, '45.8%', 55242, '78.2%'),
 ('Barking & Dagenham', 46130, '62.4%', 27750, '37.6%', 73880, '63.8%'),
 ('Barnet', 60823, '37.8%', 100210, '62.2%', 161033, '72.1%'),
 ('Barnsley', 83958, '68.3%', 38951, '31.7%', 122909, '69.9%'),
 ('Barrow-in-Furness', 21867